<a href="https://colab.research.google.com/github/Sandwhaletree/2023.05_Tibame/blob/main/%E9%80%B2%E9%9A%8EDL_%E6%9D%8E%E6%99%BA%E6%8F%9A_Class/003_CNN_models_PyTorch_0717.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torchsummary
import torchvision

torch.__version__

#### AlexNet:
https://pytorch.org/vision/stable/_modules/torchvision/models/alexnet.html#alexnet

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [ ]:
model = AlexNet()

In [ ]:
print(model)

In [ ]:
torchsummary.summary(model, (3, 224, 224))

#### VGG
https://pytorch.org/vision/stable/_modules/torchvision/models/vgg.html

In [ ]:
class VGG(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Block 5
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes),
        )
    class VGG2(nn.Module) :
        def_init_(self, num_classes=1000):
        super(),__init__()
        self.features = nn.Sequential(
        Self.flatten = nn.Sequential(
            nn.AdaptiveAvgpoo12d(1),
            nn.Flatten(),
        )
        self.classifier = n.Sequential(
        nn.Linear(512, 4096),
        nn.ReLU(),
        nn.Linear (4096, 4096),
        nn .ReLU(),
        nn.Linear(4096, num classes),
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [ ]:
model = VGG()

In [ ]:
print(model)

In [ ]:
torchsummary.summary(model, (3, 224, 224))

#### Network in Network: Inception

In [ ]:
# Basic Conv block
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1,
                 padding=0,
                 bn=False,
                 activation=True):
        super().__init__()
        self.bn = nn.BatchNorm2d(num_features=out_channels) if bn else None
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size,
                              stride=stride,
                              padding=padding)
        self.activation = nn.ReLU() if activation else None
    def forward(self, x):
        x = self.conv(x)
        if self.bn:
            x = self.bn(x)
        if self.activation:
            return self.activation(x)
        else:
            return x

# InceptionV1 module
class Inception(nn.Module):
    def __init__(
        self,
        in_channels,
        ch1x1,
        ch3x3red, ch3x3,
        ch5x5red, ch5x5,
        pool_proj):
        super().__init__()
        # branch 1x1
        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)
        # branch 3x3
        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1), # reduce chanels by 1x1 conv
            BasicConv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )
        # branch 5x5
        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1), # reduce chanels by 1x1 conv
            BasicConv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )
        # branch Pool
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            BasicConv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        # concatenate feature dims
        outputs = [branch1, branch2, branch3, branch4]
        for b in outputs: print(b.shape)
        return torch.cat(outputs, dim=1)

#### GoogLeNet (Inception v1)

https://github.com/pytorch/vision/blob/master/torchvision/models/googlenet.py

In [ ]:
model = Inception(in_channels=192,
                  ch1x1=64,
                  ch3x3red=96, ch3x3=128,
                  ch5x5red=16, ch5x5=32,
                  pool_proj=32)

In [ ]:
torchsummary.summary(model, (192, 64, 64))

In [ ]:
print('\nInception block output: ', model(torch.rand(1, 192, 64, 64)).shape)

# ResNet
https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

https://pytorch.org/hub/pytorch_vision_resnet/


In [ ]:
class IdentityBlock(nn.Module):
    def __init__(self, in_dims, out_dims, kernel_size, stride=1):
        super().__init__()
        filters1, filters2, filters3 = out_dims
        self.conv1 = BasicConv2d(in_dims, filters1, kernel_size=1, stride=stride,
                                 bn=True)
        self.conv2 = BasicConv2d(filters1, filters2, kernel_size, padding='same', bn=True)
        self.conv3 = BasicConv2d(filters2, filters3, kernel_size=1, bn=True, activation=False)
        self.relu = nn.ReLU()

    def forward(self, x):
        shortcut = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x += shortcut # Add (x, shortcut) tensor
        x = self.relu(x)
        return x

class ConvBlock(nn.Module):
    def __init__(self, in_dims, out_dims, kernel_size, stride=1):
        super().__init__()
        filters1, filters2, filters3 = out_dims
        self.conv1 = BasicConv2d(in_dims, filters1, kernel_size=1, stride=stride,
                                 bn=True)
        self.conv2 = BasicConv2d(filters1, filters2, kernel_size, padding='same', bn=True)
        self.conv3 = BasicConv2d(filters2, filters3, kernel_size=1, bn=True, activation=False)
        self.relu = nn.ReLU()
        self.skip_branch = BasicConv2d(in_dims, filters3, kernel_size=1,
                                       stride=stride,
                                       bn=True, activation=False)
    def forward(self, x):
        shortcut = self.skip_branch(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x += shortcut # Add (x, shortcut) tensor
        x = self.relu(x)
        return x

In [ ]:
# The identity block is the block that has no conv layer at shortcut.
resblock1 = IdentityBlock(in_dims=256, out_dims=[64, 64, 256], kernel_size=3, stride=1)
torchsummary.summary(resblock1, (256, 64, 64))

In [ ]:
# A block that has a conv layer at shortcut.
resblock2 = ConvBlock(in_dims=256, out_dims=[64, 64, 256], kernel_size=3, stride=2)
torchsummary.summary(resblock2, (256, 64, 64))

In [ ]:
class ResNetSimple(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.features = nn.Sequential(
            nn.ConstantPad2d(padding=3, value=0),
            BasicConv2d(3, 64, kernel_size=7, stride=2, bn=True),
            nn.ConstantPad2d(padding=1, value=0),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # S1
            ConvBlock(in_dims=64, out_dims=[64, 64, 256], kernel_size=3),
            IdentityBlock(256, [64, 64, 256], 3),
            IdentityBlock(256, [64, 64, 256], 3),
            # S2
            ConvBlock(256, [128, 128, 512], 3, stride=2),
            IdentityBlock(512, [128, 128, 512], 3),
            IdentityBlock(512, [128, 128, 512], 3),
            IdentityBlock(512, [128, 128, 512], 3),
            IdentityBlock(512, [128, 128, 512], 3),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.features(x) # (bs, 512, 7, 7)
        x = self.avgpool(x) # (bs, 512, 1, 1)
        x = torch.flatten(x, 1) # (bs, 512)
        x = self.classifier(x)
        return x

In [ ]:
model = ResNetSimple()

In [ ]:
torchsummary.summary(model, (3, 224, 224))

#### [torchvision.models](https://pytorch.org/vision/stable/models.html) : Classification

*   AlexNet
*   VGG
*   ResNet
*   SqueezeNet
*   DenseNet
*   Inception v3
*   GoogLeNet (InceptionV1)
*   ShuffleNet v2
*   MobileNetV2
*   MobileNetV3
*   ResNeXt
*   Wide ResNet
*   MNASNet


### Data Preprocess

All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel **RGB** images of shape **(3 x H x W)**, where H and W are expected to be **at least 224**. The images have to be loaded in to a range of **[0, 1]** and then normalized using **mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]**. You can use the following transform to normalize:

In [ ]:
model = torchvision.models.resnet18()
# model = torchvision.models.convnext_tiny()

In [ ]:
print(model)

In [ ]:
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

# Initialize the Weight Transforms
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()

# Apply it to the input image
img_transformed = preprocess(img)

In [ ]:
print(model)

#### Replace FC layer

In [ ]:
model.fc = nn.Linear(2048, 5)

In [ ]:
# model.fc = nn.Linear(in_features=512, out_features=6)
# print(model)

In [ ]:
inputs = torch.randn(2, 3, 224, 224)
outputs = model(inputs)
print (outputs.shape)

In [ ]:
model.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(in_features=768, out_features=6),
)

#### Model summary

In [ ]:
torchsummary.summary(model, (3, 224, 224))

#### Simple inference

In [ ]:
from urllib.request import urlretrieve
url ="https://github.com/Sandwhaletree/2023.05_Tibame/blob/main/%E9%80%B2%E9%9A%8EDL_%E6%9D%8E%E6%99%BA%E6%8F%9A_Class/Img/whale.jpg?raw=true"
urlretrieve(url, "whale.jpg")

In [ ]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

img = read_image("whale.jpg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

In [ ]:
print(model.features[0][1])

In [ ]:
# Print module by module
for i, m in enumerate(model.children()):
    print(i, m)

In [ ]:
# Print layer by layer
for i, m in enumerate(model.modules()):
    print(i, m)